In [3]:
import os
import s3fs
import pandas as pd
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]

fs = s3fs.S3FileSystem(
    anon=True,
    client_kwargs={"endpoint_url": S3_ENDPOINT_URL}
)

with fs.open("renan/diffusion/dvf.csv", mode="rb") as f:
    df = pd.read_csv(f)

/tmp/ipykernel_87512/2518782726.py:12: DtypeWarning: Columns (10,12,14,16,17,18,20,22,24,26,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)


In [4]:

#print(df.head())
print(df['code_commune'])
print(df.shape)
#df_clean = df.dropna(subset=['id_mutation', 'code_commune'])
#print(df_clean.shape)
#df_clean=df : aucune valeur manquante de commune ou d'id

problematic_ids = df.groupby('id_mutation')['code_commune'].nunique()
problematic_ids = problematic_ids[problematic_ids > 1].index

print(problematic_ids.shape)
print('2020-2' in problematic_ids)
print('2020-7' in problematic_ids)

df_clean = df[~df['id_mutation'].isin(problematic_ids)]
print(df_clean.shape)


0            1367
1            1125
2            1125
3            1125
4            1408
            ...  
20102734    75113
20102735    75105
20102736    75105
20102737    75115
20102738    75115
Name: code_commune, Length: 20102739, dtype: object
(20102739, 40)
(70086,)
True
False
(19072749, 40)


In [22]:
def strtoint(valeur):
    if isinstance(valeur, str):
        return int(valeur.strip())
    else:
        # Si c'est déjà un int, float, None, etc., on le retourne
        return valeur

print(strtoint('433'))

df_clean.loc[:,'code_commune'] = df_clean['code_commune'].apply(strtoint)

print(df_clean['code_commune'].dtype)

433
object


In [23]:
# Masque booléen : vérifie si l'élément (x) est de type entier
masque_entier = df_clean['code_commune'].apply(lambda x: isinstance(x, int))

print(masque_entier)

# Application du masque pour filtrer le DataFrame
df_clean_int = df_clean[masque_entier].copy()


0           True
14          True
15          True
16          True
17          True
            ... 
20102734    True
20102735    True
20102736    True
20102737    True
20102738    True
Name: code_commune, Length: 19072749, dtype: bool


In [35]:
ventes_par_commune=df_clean.groupby("code_commune")["id_mutation"].nunique().reset_index()

print(ventes_par_commune)

print(ventes_par_commune['code_commune'])

print(ventes_par_commune['id_mutation'].max())

print(ventes_par_commune['id_mutation'].idxmax())

print(ventes_par_commune['id_mutation'][11458])

print(ventes_par_commune['code_commune'][11458])



       code_commune  id_mutation
0              1001           71
1              1002           41
2              1004         1617
3              1005          259
4              1006           30
...             ...          ...
33008         97420          796
33009         97421          273
33010         97422         4140
33011         97423          453
33012         97424          361

[33013 rows x 2 columns]
0         1001
1         1002
2         1004
3         1005
4         1006
         ...  
33008    97420
33009    97421
33010    97422
33011    97423
33012    97424
Name: code_commune, Length: 33013, dtype: int64
60398
11458
60398
31555


In [25]:
# Masque booléen : vérifie si l'élément (x) est de type entier
masque_entier = ventes_par_commune['code_commune'].apply(lambda x: isinstance(x, int))



print(masque_entier)

# Application du masque pour filtrer le DataFrame
ventes_par_commune_int = ventes_par_commune[masque_entier]

masque_str = ventes_par_commune['code_commune'].apply(lambda x: isinstance(x, str))

print(masque_str)




0        True
1        True
2        True
3        True
4        True
         ... 
33008    True
33009    True
33010    True
33011    True
33012    True
Name: code_commune, Length: 33013, dtype: bool
0        False
1        False
2        False
3        False
4        False
         ...  
33008    False
33009    False
33010    False
33011    False
33012    False
Name: code_commune, Length: 33013, dtype: bool
